In [139]:
import numpy as np
import pandas as pd
import time as tt

def propagate_keplerian(input_oe, param):
    """
    Propagates the orbital elements using Kepler's equation.
    
    Args:
        input_oe (numpy.ndarray): Initial orbital elements [a, e, i, RAAN, omega, M].
        param (dict): Dictionary containing parameters like mu, t, and t_0.
    
    Returns:
        numpy.ndarray: Propagated orbital elements [a, e, i, RAAN, omega, M].
    """
    mu = param['mu']
    t = param['t']
    t_0 = param['t_0']
    deg = np.pi/180

    # Extract initial orbital elements
    a = input_oe[:, 0]
    e = input_oe[:, 1]
    i = input_oe[:, 2]
    RAAN = input_oe[:, 3]
    omega = input_oe[:, 4]
    M0 = input_oe[:, 5]

    # Mean motion
    n = np.sqrt(mu) * a**(-3/2)

    M_no_wrap = M0 + n * (t - t_0)  # Mean anomaly at time t
    # M = M_no_wrap
    # if M_no_wrap > 2 * np.pi:
    M = np.mod(M_no_wrap, 2 * np.pi)  # Wrap to [0, 2*pi]
    # else: 
    #     M = M_no_/wrap

    # Update orbital elements
    propagated_oe = np.column_stack([a, e, i, RAAN, omega, M]) #np.mod(nu, 2 * np.pi)])
    return propagated_oe

def analytic_prop_vec_MOD(input_oe, param):
    # mean orbital elements propagation

    errors = np.zeros((input_oe.shape[0], 1))

    re = param['req']
    J2 = param['j2']
    mu = param['mu']

    a_0 = input_oe[:, 0]
    rho_0 = param['rho']
    C_0 = np.maximum(0.5 * param['Bstar'] * rho_0, 1e-20) # Corrected Bstar usage
    k2_over_mu = J2 * re**2 / 2  # k2 = mu*J2*re^2/2

    t = param['t']
    t_0 = param['t_0']

    e_0 = input_oe[:, 1]
    inc_0 = input_oe[:, 2]
    bigO_0 = input_oe[:, 3]
    omega_0 = input_oe[:, 4]
    Mo_0 = input_oe[:, 5]

    c = np.cos(inc_0)
    c_sq = c**2

    n_0 = np.sqrt(mu) * a_0**(-3/2)

    alpha0_sq = (e_0 / np.sqrt(a_0))**2

    beta_0 = (np.sqrt(3)/2) * e_0

    tan_atan_beta0 = np.maximum(np.tan(np.arctan(beta_0) - beta_0 * n_0 * a_0 * C_0 * (t - t_0)), 0)
    # aaa = np.tan(np.arctan(beta_0) - beta_0 * n_0 * a_0 * C_0 * (t - t_0))
    # print(f"tan_atan_beta0: {aaa[0]:.15f}")
    a = (a_0 / beta_0**2) * tan_atan_beta0**2
    e = (2/np.sqrt(3)) * tan_atan_beta0

    check_beta = beta_0 == 0
    print(beta_0)
    if np.any(check_beta):  # Avoid a=NaN when beta_0=0
        a0_beta = a_0[check_beta]
        a[check_beta] = a0_beta * (1 - C_0[check_beta] * n_0[check_beta] * a0_beta * (t - t_0))

    # Compute some variables to avoid repetition of operations
    a_sq = a**2
    four_thirds_over_a_cb = 4/3 / (a_sq * a)
    a0_sq = a_0**2
    four_thirds_over_a0_cb = 4/3 / (a0_sq * a_0)
    alpha0sq_over_asq = alpha0_sq / a_sq
    alpha0sq_over_a0sq = alpha0_sq / a0_sq

    # orig: 
    # Mo = ( (0.5/a - 0.5/a_0) + 3/8 * alpha0_sq * np.log(a/a_0) ) / C_0 + 3*k2_over_mu/16*(3*c_sq-1) * (1.5 * (alpha0sq_over_asq - alpha0sq_over_a0sq) + (four_thirds_over_a_cb - four_thirds_over_a0_cb) ) / C_0 + Mo_0
    Mo = ( -(0.5/a - 0.5/a_0) - 3/8 * alpha0_sq * np.log(a/a_0) ) / C_0 + 3*k2_over_mu/16*(3*c_sq-1) * (1.5 * - (alpha0sq_over_asq - alpha0sq_over_a0sq) - (four_thirds_over_a_cb - four_thirds_over_a0_cb) ) / C_0 + Mo_0

    # orig: 
    # five_a0sq_over2_tau2_plus_4thirds_over_tau3_over_C0 = (2.5 * (alpha0sq_over_asq - alpha0sq_over_a0sq) + (four_thirds_over_a_cb - four_thirds_over_a0_cb) ) / C_0
    five_a0sq_over2_tau2_plus_4thirds_over_tau3_over_C0 = (2.5 * - (alpha0sq_over_asq - alpha0sq_over_a0sq) - (four_thirds_over_a_cb - four_thirds_over_a0_cb) ) / C_0

    omega = 3*k2_over_mu/16 * (5*c_sq - 1) * five_a0sq_over2_tau2_plus_4thirds_over_tau3_over_C0 + omega_0

    bigO = -3*k2_over_mu/8 * c * five_a0sq_over2_tau2_plus_4thirds_over_tau3_over_C0 + bigO_0

    out_oe = np.column_stack([a, e, np.mod(inc_0, 2*np.pi), np.mod(bigO, 2*np.pi), np.mod(omega, 2*np.pi), np.mod(Mo, 2*np.pi)])

    not_real = ~np.isreal(inc_0) | ~np.isreal(bigO) | ~np.isreal(omega) | ~np.isreal(Mo)
    errors[not_real] = 1

    out_oe[not_real, :] = input_oe[not_real, :]

    return out_oe, errors

In [140]:
mu = 398600  # gravitational parameter in km^3/s^2
J2 = 1082.63e-6  # J2 coefficient
RE = 6378  # Earth's radius in km
cd = 2.2  # drag coefficient
A = 1 * 1e-6 # cross-sectional area in m^2 or *1e-6 for [km^2]
m = 100  # mass of the satellite in kg
B = cd*A/m
rho0 = 1.944192087333296e-06  # kg/km^3 (conversion is: [kg/m^3] or *1e9 [kg/km^3])

from_matlab1 = [7507.6290061995,
         0.116640408348328,
         0.785703852208281,
         0.174930790861768,
         0.524936882311218,
          1.00779162406823]

from_matlab2 = [6782.58140364874,
      0.000220331022356249,
         0.785735620099787,
         0.174706236385514,
          2.12610210614672,
        -0.380710999149193]

from_matlab3 = [7511.88131938137,
         0.117050755060748,
          1.91966574138367,
         0.174340452911236,
         0.534316518462145,
         0.999012283321003]

from_matlab4 = [7011.49667759347,
        0.0539825302809328,
          1.91965033679686,
          3.66504862488471,
         0.548835656769462,
          1.09760204144139]

from_matlab5 = [7009.74059936841,
         0.025450330656246,
          1.91969154363248,
          1.92003970020848,
          2.22302522140705,
        -0.958973564267631]

from_matlab = from_matlab1

# Initial Mean Orbital Elements (from MATLAB)
a0 = from_matlab[0] # initial semi-major axis in km
e0 = from_matlab[1] # initial eccentricity
i0 = from_matlab[2]  # initial inclination in radians
Omega0 = from_matlab[3]  # initial right ascension of ascending node in radians
omega0 = from_matlab[4]  # initial argument of perigee in radians
M0 = from_matlab[5]  # initial mean anomaly in radians
input_oe = np.array([[a0, e0, i0, Omega0, omega0, M0]])

t0 = pd.Timestamp('2023-01-01 00:00:00')  # initial time
t = pd.Timestamp('2023-01-01 00:00:00.1')  # final time
dt = (t - t0).total_seconds()

param = {
    'req': RE,    
    'j2': J2 * 0,
    'mu': mu,
    'Bstar': B * 0, 
    'rho': rho0, 
    't_0': 0, 
    't': dt           
}

# Propagate the mean orbital elements
out_oe, _ = analytic_prop_vec_MOD(input_oe,param)

# Print the results
deg = np.pi/180  # convert radian to degree, if necessary

print("Initial Mean Orbital Elements:")
print(f"  sma: {input_oe[:,0][0]:.3f} km")
print(f"  e: {input_oe[:,1][0]:.6f}")
print(f"  incl: {input_oe[:,2][0]/deg:.6f} deg")
print(f"  RAAN: {input_oe[:,3][0]/deg:.6f} deg")
print(f"  arg_per: {input_oe[:,4][0]/deg:.6f} deg")
print(f"  mean_anom: {input_oe[:,5][0]/deg:.6f} deg")

print("\n")  # Print a blank line

print("Final Mean Orbital Elements:")
print(f"  sma: {out_oe[:,0][0]:.3f} km")
print(f"  e: {out_oe[:,1][0]:.6f}")
print(f"  incl: {out_oe[:,2][0]/deg:.6f} deg")
print(f"  RAAN: {out_oe[:,3][0]/deg:.6f} deg")
print(f"  arg_per: {out_oe[:,4][0]/deg:.6f} deg")
print(f"  mean_anom: {out_oe[:,5][0]/deg:.6f} deg")
# Note: The above code assumes that the atmospheric density rho0 is constant.

# # output using from_matlab1 as initial conditions
# Initial Mean Orbital Elements:
#   sma: 7507.629 km
#   e: 0.116640
#   incl: 45.017515 deg
#   RAAN: 10.022796 deg
#   arg_per: 30.076668 deg
#   mean_anom: 57.742207 deg
# Final Mean Orbital Elements:
#   sma: 7507.629 km
#   e: 0.116640
#   incl: 45.017515 deg
#   RAAN: 5.933295 deg
#   arg_per: 34.411144 deg
#   mean_anom: 183.204813 deg

# # output using from_matlab2 as initial conditions
# Initial Mean Orbital Elements:
#   sma: 6782.581 km
#   e: 0.000220
#   incl: 45.019335 deg
#   RAAN: 10.009930 deg
#   arg_per: 121.816677 deg
#   mean_anom: -21.813133 deg
# Final Mean Orbital Elements:
#   sma: 6782.582 km
#   e: 0.000220
#   incl: 45.019335 deg
#   RAAN: 4.330546 deg
#   arg_per: 127.835829 deg
#   mean_anom: 175.351075 deg

[0.10101356]
Initial Mean Orbital Elements:
  sma: 7507.629 km
  e: 0.116640
  incl: 45.017515 deg
  RAAN: 10.022796 deg
  arg_per: 30.076668 deg
  mean_anom: 57.742207 deg


Final Mean Orbital Elements:
  sma: 7507.629 km
  e: 0.116640
  incl: 45.017515 deg
  RAAN: 10.022796 deg
  arg_per: 30.076668 deg
  mean_anom: 57.742207 deg


In [144]:
from_matlab1 = [6682.70631579479,
      0.000230723598916234,
         0.785749989047153,
          0.17469421897602,
          2.06451203498031,
        -0.336578490605857]

from_matlab2 = [6782.58140364874,
      0.000220331022356249,
         0.785735620099787,
         0.174706236385514,
          2.12610210614672,
        -0.380710999149193]

from_matlab = from_matlab2

# Initial Mean Orbital Elements (from MATLAB)
a0 = from_matlab[0] # initial semi-major axis in km
e0 = from_matlab[1] # initial eccentricity
i0 = from_matlab[2]  # initial inclination in radians
Omega0 = from_matlab[3]  # initial right ascension of ascending node in radians
omega0 = from_matlab[4]  # initial argument of perigee in radians
M0 = from_matlab[5]  # initial mean anomaly in radians
input_oe = np.array([[a0, e0, i0, Omega0, omega0, M0]])

t0 = pd.Timestamp('2023-01-01 00:00:00')  # initial time
t = pd.Timestamp('2023-01-01 00:00:01')  # final time
dt = (t - t0).total_seconds()

param = {
    'req': RE,    
    'j2': J2 * 0,
    'mu': mu,
    'Bstar': B * 0, 
    'rho': rho0, 
    't': dt, 
    't_0': 0,        
}

# Propagate the mean orbital elements
start_time = tt.time()
out_oe, _ = analytic_prop_vec_MOD(input_oe, param)
end_time = tt.time()

print(f"Cpu time (analytical): {end_time - start_time:.6f} seconds")

start_time = tt.time()
out_oe_kep = propagate_keplerian(input_oe, param)
end_time = tt.time()

print(f"Cpu time (kepler): {end_time - start_time:.6f} seconds\n")

# Print the results
deg = np.pi/180  # convert radian to degree, if necessary

print("Initial Mean Orbital Elements:")
print(f"  sma: {input_oe[:,0][0]:.3f} km")
print(f"  e: {input_oe[:,1][0]:.6f}")
print(f"  incl: {input_oe[:,2][0]/deg:.6f} deg")
print(f"  RAAN: {input_oe[:,3][0]/deg:.6f} deg")
print(f"  arg_per: {input_oe[:,4][0]/deg:.6f} deg")
print(f"  mean_anom: {input_oe[:,5][0]/deg:.6f} deg\n")

# print("\n")  # Print a blank line

print("Final Mean Orbital Elements:")
print(f"  sma: {out_oe[:,0][0]:.3f} km")
print(f"  e: {out_oe[:,1][0]:.6f}")
print(f"  incl: {out_oe[:,2][0]/deg:.6f} deg")
print(f"  RAAN: {out_oe[:,3][0]/deg:.6f} deg")
print(f"  arg_per: {out_oe[:,4][0]/deg:.6f} deg")
print(f"  mean_anom analytical: {out_oe[:,5][0]/deg:.6f} deg")
print(f"  mean_anom keplerian: {out_oe_kep[:,5][0]/deg:.6f} deg")


[0.00019081]
Cpu time (analytical): 0.000953 seconds
Cpu time (kepler): 0.000053 seconds

Initial Mean Orbital Elements:
  sma: 6782.581 km
  e: 0.000220
  incl: 45.019335 deg
  RAAN: 10.009930 deg
  arg_per: 121.816677 deg
  mean_anom: -21.813133 deg

Final Mean Orbital Elements:
  sma: 6782.581 km
  e: 0.000220
  incl: 45.019335 deg
  RAAN: 10.009930 deg
  arg_per: 121.816677 deg
  mean_anom analytical: 55.837124 deg
  mean_anom keplerian: 338.251625 deg
